In [60]:
import pandas as pd
from tqdm import tqdm
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

DATA_ROOT = '../data'

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [61]:
# Read in the data
df_db = pd.read_csv(f'{DATA_ROOT}/raw/DS_train/DS_train(2020-06--2022-06-01).csv', engine="pyarrow", delimiter='\t')
df_db.shape

(69173886, 4)

In [62]:
df_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69173886 entries, 0 to 69173885
Data columns (total 4 columns):
 #   Column        Dtype         
---  ------        -----         
 0   WebPriceId    int64         
 1   DateObserve   datetime64[ns]
 2   StockStatus   object        
 3   CurrentPrice  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 2.1+ GB


In [63]:
DATE_COLUMN = "DateObserve"
PRICE_COLUMN = "CurrentPrice"
ID_COLUMN = "WebPriceId"
STOCK_STATUS_COLUMN = "StockStatus"

In [64]:
# Remove webpriceids with less than 10 observations
# df_db = df_db.groupby(ID_COLUMN).filter(lambda x: len(x) > 10)

In [65]:
def select_data_in_between(df: pd.DataFrame, date_start: str = None, date_end: str = None) -> pd.DataFrame:
    """
    Select data between two dates
    """
    if date_start is None:
        date_start = df[DATE_COLUMN].min()
    if date_end is None:
        date_end = df[DATE_COLUMN].max()
    return df[(df[DATE_COLUMN] >= date_start) & (df[DATE_COLUMN] <= date_end)]

In [66]:
def find_nearest_to_date_in_series(series: pd.Series, date: str) -> pd.Series:
    """
    Find nearest date in a series
    """
    return series.iloc[(series - pd.to_datetime(date)).abs().argsort()[:1]]

In [72]:
# Remove webpriceids with less than 10 observations
df_db = df_db[:100000000].groupby(ID_COLUMN).filter(lambda x: len(x) > 10)

In [113]:
# Реализация бейзлайна
def baseline_prediction(df: pd.DataFrame, date: str, k: int = 5000) -> float:
    """
        Предсказание ИПЦ для конца месяца, указанного в date,
        на основании данных до date. Среднее по ИПЦ всех отдельных
        товаров, рассчитывваемое по формуле

        ИПЦ = Цена в date / Цена в date - 1, но месяц назад

    Arguments:
        df: pd.DataFrame - все данные
        date: str - имеет формат "ГГГГ-ММ-ДД"
        k: int - лимит на количество товаров, по которым считается среднее

    Return:
        IPC: float - предсказанная ИПЦ за месяц

    """


    # Разбить date на текущий месяц, день и год
    year, month, day = date.split('-')

    # Выбрать данные от месяца назад до date, использовать pd.Timedelta
    df_month = select_data_in_between(df, date_start=pd.to_datetime(date) - pd.Timedelta(days=31), date_end=date)
    df_month = df_month[df_month[STOCK_STATUS_COLUMN] == "InStock"]

    # Выбрать те WebPriceId, записи о которых есть в df_month
    ids = df_month[ID_COLUMN].unique()

    # Отсеить WebPriceId, у которых меньше 3 записей за этот период
    ids = df_month.groupby(ID_COLUMN).filter(lambda x: len(x) > 3)[ID_COLUMN].unique()

    ipc_frame = pd.DataFrame()
    # Для каждого товара в промежутке посчитать ИПЦ
    total_drops = 0
    i = 0
    for web_price_id in tqdm(ids, total=min(len(ids), k)):
        # Если достигнут лимит, то прервать цикл
        if i >= k + total_drops:
            break
        i += 1

        # Выбрать данные по WebPriceId
        df_id = df_month[df_month[ID_COLUMN] == web_price_id]

        # Найти ближайшую стоимость к date и к date - 1 месяц
        nearest_date = find_nearest_to_date_in_series(df_id[DATE_COLUMN], date)
        nearest_date_prev = find_nearest_to_date_in_series(df_id[DATE_COLUMN], pd.to_datetime(date) - pd.Timedelta(days=29))

        # Если nearest_date или nearest_date_prev пустые, то пропустить товар
        if nearest_date.empty or nearest_date_prev.empty:
            continue

        # Отсеить товар, если между краевыми датами меньше 25 дней
        days_in_between = (pd.to_datetime(nearest_date.values[0]) - pd.to_datetime(nearest_date_prev.values[0])).days
        if days_in_between < 20:
            # print('drop: ', days_in_between)
            total_drops += 1
            continue

        # Выбрать стоимость по ближайшей дате
        price = df_id[df_id[DATE_COLUMN] == nearest_date.values[0]][PRICE_COLUMN].values[0]
        price_prev = df_id[df_id[DATE_COLUMN] == nearest_date_prev.values[0]][PRICE_COLUMN].values[0]
        interval = days_in_between

        # Посчитать ИПЦ
        ipc_frame = pd.concat([ipc_frame, pd.DataFrame([[web_price_id, price, price_prev, interval]])])

        # Поправка на интервал между датами
        # ipc = ipc ** (30 / days_in_between)

    try:
        ipc_frame['ipc'] = ipc_frame.apply(lambda x: (x[1] / x[2]) * (30/interval) - 1, axis=1)
        return ipc_frame['ipc'].mean(), total_drops
    except:
        return 0, -1

baseline_prediction(df_db, '2021-10-01', k=1000)


100%|██████████| 798/798 [00:02<00:00, 342.17it/s]


(0.5281096688537906, 453)

In [96]:
# Evaluate baseline

# Load test data
def read_xlsx(path):
    df = pd.read_excel(path, engine='openpyxl')
    df = df.transpose()
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    # drop index
    df = df.reset_index(drop=True)

    return df

# read data
df_y_train = read_xlsx(f"../data/raw/Y_train.xlsx")
df_y_train.head()


"ИПЦ, мом",Период,Целевой показатель
0,2020-06-01 00:00:00,0.28
1,2020-07-01 00:00:00,-0.065
2,2020-08-01 00:00:00,-0.005
3,2020-09-01 00:00:00,0.315
4,2020-10-01 00:00:00,0


In [97]:
# Convert Период to datetime
df_y_train['Период'] = pd.to_datetime(df_y_train['Период'])
df_y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Период              24 non-null     datetime64[ns]
 1   Целевой показатель  24 non-null     object        
dtypes: datetime64[ns](1), object(1)
memory usage: 512.0+ bytes


In [107]:
print(df_y_train['Период'])

0    2020-06-01
1    2020-07-01
2    2020-08-01
3    2020-09-01
4    2020-10-01
5    2020-11-01
6    2020-12-01
7    2021-01-01
8    2021-02-01
9    2021-03-01
10   2021-04-01
11   2021-05-01
12   2021-06-01
13   2021-07-01
14   2021-08-01
15   2021-09-01
16   2021-10-01
17   2021-11-01
18   2021-12-01
19   2022-01-01
20   2022-02-01
21   2022-03-01
22   2022-04-01
23   2022-05-01
Name: Период, dtype: datetime64[ns]


In [116]:
df_y_train['Целевой показатель']


0      0.28
1    -0.065
2    -0.005
3     0.315
4         0
5     0.375
6      0.35
7      0.38
8     0.685
9     0.255
10     0.13
11    0.485
12     0.25
13   -0.135
14     0.35
15    0.345
16    0.675
17     0.74
18    1.255
19    0.355
20    1.335
21    3.555
22     1.89
23     1.56
Name: Целевой показатель, dtype: object

In [115]:

list_preds = []

i=0
for date in df_y_train['Период']:
    print(date.strftime('%Y-%m-%d'))
    list_preds.append(baseline_prediction(df_db, date.strftime('%Y-%m-%d'), k=1000))
    print(f"check {i}")
    

2020-06-01


0it [00:00, ?it/s]


(0, -1)
2020-07-01


100%|██████████| 4/4 [00:00<00:00, 418.08it/s]


(0, -1)
2020-08-01


...

(0.2584344821277295, 120)
2020-11-01


100%|██████████| 408/408 [00:01<00:00, 307.03it/s]


(0.26521559937890504, 142)
2020-12-01


 60%|██████    | 252/418 [00:00<00:00, 264.05it/s]


KeyboardInterrupt: 